In [ ]:
import pandas as pd
import random

In [ ]:
p = 0.01

df = pd.read_csv(
    'MICRODADOS_ENEM_ESCOLA.csv',
    header=0,
    encoding = "ISO-8859-1",
    skiprows=lambda i: i > 0 and random.random() > p,
    sep=';'
)

df.head()